In [1]:
import sys
import getopt
import os
import json
import argparse
import uuid
import pandas as pd
from collections import defaultdict

from gnps_index import *
from numba.typed import List
import numpy as np
from pyteomics import mgf
from library_search_indexed import read_mgf


In [82]:
spectra_path = "/home/user/LabData/Reza/Code/LibrarySearch_Workflow/data/spectra/selected.mgf"
library_path = "/home/user/LabData/Reza/Code/LibrarySearch_Workflow/data/libraries/rest.mgf"

In [83]:
spectrum_mgf = read_mgf(spectra_path)
# print(spectrum_mgf[0].keys())

library_mgf = read_mgf(library_path)

In [84]:
def convert_single_spectrum_to_spectrum_list(data_dict):
    mz = [float(x[0]) for x in data_dict['peaks']]
    intensity = [float(x[1]) for x in data_dict['peaks']]
    precursor_mz = float(data_dict.get('PEPMASS', 0.0))
    precursor_charge = data_dict.get('CHARGE', data_dict.get('charge', 1))
    precursor_charge = abs(int(precursor_charge))  # Ensure charge is positive
    # print("log", f"Converted spectrum with {len(mz)} peaks, precursor_mz: {precursor_mz}, precursor_charge: {precursor_charge}")
    # print("log", f"mz: {mz[:5]}, intensity: {intensity[:5]}")
    mz = np.asarray(mz, dtype=np.float32)
    intensity = np.asarray(intensity, dtype=np.float32)
    return filter_peaks_optimized(mz, intensity, precursor_mz, precursor_charge)

spectrum_list = [convert_single_spectrum_to_spectrum_list(spec) for spec in spectrum_mgf]
library_list = [convert_single_spectrum_to_spectrum_list(spec) for spec in library_mgf]

In [6]:
df = pd.DataFrame(library_mgf)
df.head(5)

,PEPMASS,SPECTRUMID,SCANS,peaks,CHARGE,MSLEVEL,SOURCE_INSTRUMENT,FILENAME,SEQ,IONMODE,...,NAME,PI,DATACOLLECTOR,SMILES,INCHI,INCHIAUX,PUBMED,SUBMITUSER,LIBRARYQUALITY,PRECURSOR_MZ
0,223.051,CCMSLIB00010101988,506,"[[66.79527, 2.340182], [74.76895, 2.3136747], ...",-1,2,LC-ESI-Orbitrap,tharwood/20220308_JGI-AK-TH_TN_507992_PlantStd...,*..*,Negative,...,Phenazine-1-carboxylic acid CollisionEnergy:10...,Trent Northen,JGI,C1=CC=C2C(=C1)N=C3C=CC=C(C3=N2)C(=O)O,"""InChI=1S/C13H8N2O2/c16-13(17)8-4-3-7-11-12(8)...",N/A,N/A,mwang87,3,223.051
1,223.051,CCMSLIB00010101989,503,"[[74.99028, 16.873964], [79.957, 4.377357], [8...",-1,2,LC-ESI-Orbitrap,tharwood/20220308_JGI-AK-TH_TN_507992_PlantStd...,*..*,Negative,...,Phenazine-1-carboxylic acid CollisionEnergy:20...,Trent Northen,JGI,C1=CC=C2C(=C1)N=C3C=CC=C(C3=N2)C(=O)O,"""InChI=1S/C13H8N2O2/c16-13(17)8-4-3-7-11-12(8)...",N/A,N/A,mwang87,3,223.051
2,405.182,CCMSLIB00010101990,603,"[[65.328674, 20.255243], [92.43383, 29.783403]...",-1,2,LC-ESI-Orbitrap,tharwood/20220308_JGI-AK-TH_TN_507992_PlantStd...,*..*,Negative,...,indole-3-butyric acid CollisionEnergy:102040 2M-H,Trent Northen,JGI,C1=CC=C2C(=C1)C(=CN2)CCCC(=O)O,"""InChI=1S/C12H13NO2/c14-12(15)7-3-4-9-8-13-11-...",N/A,N/A,mwang87,3,405.182
3,185.061,CCMSLIB00010101991,614,"[[52.97066, 1.8730004], [54.45359, 1.9932256],...",-1,2,LC-ESI-Orbitrap,tharwood/20220308_JGI-AK-TH_TN_507992_PlantStd...,*..*,Negative,...,1-Naphthaleneacetic acid CollisionEnergy:10204...,Trent Northen,JGI,C1=CC=C2C(=C1)C=CC=C2CC(=O)O,"""InChI=1S/C12H10O2/c13-12(14)8-10-6-3-5-9-4-1-...",N/A,N/A,mwang87,3,185.061
4,185.061,CCMSLIB00010101992,605,"[[60.311302, 4.445654], [70.33653, 3.9257047],...",-1,2,LC-ESI-Orbitrap,tharwood/20220308_JGI-AK-TH_TN_507992_PlantStd...,*..*,Negative,...,1-Naphthaleneacetic acid CollisionEnergy:20506...,Trent Northen,JGI,C1=CC=C2C(=C1)C=CC=C2CC(=O)O,"""InChI=1S/C12H10O2/c13-12(14)8-10-6-3-5-9-4-1-...",N/A,N/A,mwang87,3,185.061


In [7]:
library_list

[(array([ 66.79527 ,  74.76895 ,  79.18015 ,  86.134895,  87.82698 ,
          96.95974 , 121.07147 , 147.412   , 155.0611  , 167.06174 ,
         179.0614  , 180.06677 , 182.04811 , 183.05711 , 195.05638 ,
         197.07204 , 198.07706 , 205.01732 , 223.0284  , 223.05179 ,
         223.10103 , 224.05627 ], dtype=float32),
  array([0.03515802, 0.03495833, 0.03712093, 0.03435741, 0.03642141,
         0.05856638, 0.03914803, 0.03691255, 0.10134196, 0.08004299,
         0.4400461 , 0.04054559, 0.05396103, 0.08639131, 0.2676156 ,
         0.72641087, 0.05568074, 0.09167421, 0.28776908, 0.25768742,
         0.04711797, 0.05258903], dtype=float32),
  223.051,
  1),
 (array([ 74.99028,  79.957  ,  85.29974,  93.00113,  96.95953, 129.26405,
         149.00882, 155.06113, 167.06093, 178.05325, 179.06093, 182.04813,
         183.05539, 195.0558 , 197.07156, 198.07466, 205.01703, 206.99689,
         211.44548, 223.02745, 223.05089, 224.05705], dtype=float32),
  array([0.09447905, 0.04812081, 0.0

In [72]:

def convert_to_mamba_format(file_list):
    numba_spectra = List()
    for spec in file_list:
        numba_spectra.append((
            spec[0].astype(np.float32),
            spec[1].astype(np.float32),
            np.float32(spec[2]),
            np.int32(spec[3])
        ))
    n_spectra = len(numba_spectra)
    return numba_spectra, n_spectra

@numba.njit
def cross_library_compute_all_pairs(spectra, library, shared_entries,
                                    shifted_entries, tolerance,
                                    threshold, topk,
                                    library_min_matched_peaks):
    results = List()

    n_library = len(library)
    n_spectra = len(spectra)
    for query_idx in range(n_spectra):
        print(f"Processing query spectrum {query_idx + 1}/{n_spectra}...")
        query_spec = spectra[query_idx]
        upper_bounds = np.zeros(n_library, dtype=np.float32)
        match_counts = np.zeros(n_library, dtype=np.int32)

        # Process both shared and shifted peaks
        for peak_idx in range(len(query_spec[0])):
            mz = query_spec[0][peak_idx]
            intensity = query_spec[1][peak_idx]
            precursor_mz = query_spec[2]

            # Shared peaks processing
            shared_bin = np.int64(round(mz / tolerance))
            shifted_bin = np.int64(round((precursor_mz - mz + SHIFTED_OFFSET) / tolerance))

            # Check both shared and shifted entries
            for entries, bin_val in [(shared_entries, shared_bin),
                                     (shifted_entries, shifted_bin)]:
                for delta in ADJACENT_BINS:
                    target_bin = bin_val + delta
                    start, end = find_bin_range(entries, target_bin)
                    pos = start
                    while pos < end and entries[pos][1] <= query_idx:
                        pos += 1
                    # Find matches in this bin
                    while pos < end and entries[pos][0] == target_bin:
                        spec_idx = entries[pos][1]
                        upper_bounds[spec_idx] += intensity * entries[pos][4]
                        match_counts[spec_idx] += 1
                        pos += 1
        # print("log", f"Upper bounds for query {query_idx}: {len(upper_bounds)}")
        # Collect candidates using threshold parameter
        candidates = List()
        for lib_idx in range(n_library):
            if lib_idx == 10748:
                print(upper_bounds[lib_idx], match_counts[lib_idx])
            if (upper_bounds[lib_idx] >= threshold and
                    match_counts[lib_idx] >= library_min_matched_peaks):
                candidates.append((lib_idx, upper_bounds[lib_idx]))
        
        # print("log", f"Candidates for query {query_idx}: {len(candidates)}")
        # for item in candidates:
        #     print("log", f"Candidate: {item[0]}, Score: {item[1]}")
        # Sort candidates by upper bound score
        candidates.sort(key=lambda x: -x[1])
        # print("log", f"sorted candidates")
        print("candidates", candidates)
        # Process top candidates for exact matching
        exact_matches = List()
        for lib_idx, _ in candidates:
            # print("log", f"Processing exact match for candidate {lib_idx}...")
            target_spec = library[lib_idx]
            score, shared, shifted, num_matches = calculate_exact_score_GNPS(spectra[query_idx], target_spec, tolerance)
            # score is of type <object type:float64> print it in log as a float value human readable
            print("log", f"Score for candidate {lib_idx}:", score.item())
            if score >= threshold:
                exact_matches.append((lib_idx, score, shared, shifted, num_matches))

        # Sort and store top results
        exact_matches.sort(key=lambda x: -x[1])
        results.append((query_idx, exact_matches[:topk]))

    return results

In [31]:
spectrum_list = spectrum_list[0:1]
converted_spectrum_list, num_spectra = convert_to_mamba_format(spectrum_list)
converted_library_list, num_library = convert_to_mamba_format(library_list)

library_shared_idx = create_index(converted_library_list, False, 0.5, SHIFTED_OFFSET)
library_shifted_idx = create_index(converted_library_list, True, 0.5, SHIFTED_OFFSET)

In [73]:
matches = cross_library_compute_all_pairs(converted_spectrum_list,
                                              converted_library_list,
                                              library_shared_idx,
                                              library_shifted_idx,
                                tolerance=0.5, threshold=0.7, topk=1, library_min_matched_peaks=6)

Processing query spectrum 1/1...
1.9671087265014648 51
candidates [(10748, 1.9671087265014648), (22041, 1.7978641986846924), (1072, 1.0298961400985718), (24643, 1.0298961400985718), (19398, 0.9269737005233765), (8081, 0.9211397171020508), (1643, 0.9182926416397095), (4325, 0.9124040603637695), (1594, 0.9032070636749268), (15646, 0.895987868309021), (12920, 0.8232729434967041), (173, 0.811319887638092), (172, 0.8040243983268738), (8279, 0.8000421524047852), (10842, 0.785700798034668), (2848, 0.7791898250579834), (10818, 0.7790063619613647), (17809, 0.7613140940666199), (2055, 0.7549872398376465), (1762, 0.7492693662643433), (24739, 0.7479681968688965), (1168, 0.7479681968688965), (4534, 0.7439603209495544), (3035, 0.7284307479858398), (8876, 0.7262328863143921), (10980, 0.7216756939888), (24818, 0.7209244966506958), (1247, 0.7209244966506958), (4204, 0.7192760705947876), (6493, 0.7153799533843994), (20957, 0.7138700485229492), (12853, 0.7119367122650146), (22271, 0.7116252183914185), (1

In [91]:
spectrum_mgf[query_idx]

{'PEPMASS': 434.128,
 'SPECTRUMID': 'CCMSLIB00010124534',
 'SCANS': '22098',
 'peaks': array([[5.0090961e+01, 1.0042304e+00],
        [6.0774689e+01, 1.0781515e+00],
        [6.5412918e+01, 1.2108222e+00],
        [6.7712196e+01, 1.0006795e+00],
        [7.3937401e+01, 1.0300543e+00],
        [9.7054321e+01, 1.1549778e+00],
        [1.0305491e+02, 9.9293232e-01],
        [1.0507028e+02, 2.0065238e+00],
        [1.3006531e+02, 1.3763863e+01],
        [1.3107298e+02, 1.3079849e+00],
        [1.3502661e+02, 1.4793917e+00],
        [1.4507605e+02, 5.9763165e+00],
        [1.4807545e+02, 6.3746514e+00],
        [1.5003694e+02, 5.0033975e+00],
        [1.5707594e+02, 5.9772849e+00],
        [1.5806006e+02, 1.0315715e+01],
        [1.5808347e+02, 1.7337580e+00],
        [1.5906750e+02, 5.2806826e+00],
        [1.6007629e+02, 1.3564049e+00],
        [1.6203705e+02, 7.3391129e+01],
        [1.6405296e+02, 1.0123649e+01],
        [1.6587277e+02, 1.1204382e+00],
        [1.6907585e+02, 4.6676860e

In [79]:
for i in range(len(spectrum_mgf)):
    if int(spectrum_mgf[i]['SCANS']) == 22098:
        print("Found query spectrum at index:", i)
        query_idx = i
        break

Found query spectrum at index: 41


In [28]:
library_mgf[24925]

{'PEPMASS': 186.016,
 'SPECTRUMID': 'CCMSLIB00010126959',
 'SCANS': '24523',
 'peaks': array([[ 53.7097   ,   1.4847158],
        [ 60.5486   ,   1.4137611],
        [ 69.446    ,   1.5775278],
        [ 69.764    ,   1.4533497],
        [ 70.0293   ,  31.554653 ],
        [ 83.9781   ,   1.650699 ],
        [ 88.0398   , 999.       ],
        [ 94.6251   ,   1.5492384],
        [104.305    ,   1.3858774],
        [114.113    ,   1.4237576],
        [140.011    ,   2.9121485],
        [158.001    ,   1.7927151],
        [166.703    ,   1.3578134],
        [185.435    ,   2.3285198],
        [186.016    ,  20.187632 ],
        [186.185    ,   1.6886365]], dtype=float32),
 'CHARGE': 1,
 'MSLEVEL': '2',
 'SOURCE_INSTRUMENT': 'LC-ESI-Orbitrap',
 'FILENAME': 'tharwood/spectral_libraries/BERKELEY-LAB.mgf',
 'SEQ': '*..*',
 'IONMODE': 'Positive',
 'ORGANISM': 'BERKELEY-LAB',
 'NAME': 'O-phospho-L-serine CollisionEnergy:102040 M+H',
 'PI': 'Trent Northen',
 'DATACOLLECTOR': 'JGI',
 'SMILES': '

In [36]:
library_mgf[10748]

{'PEPMASS': 392.185,
 'SPECTRUMID': 'CCMSLIB00010112747',
 'SCANS': '10313',
 'peaks': array([[ 55.01869  ,   2.9565785],
        [ 55.37616  ,   1.9496881],
        [ 56.05012  ,   2.8142047],
        [ 57.07058  ,  11.240429 ],
        [ 58.99576  ,  13.415787 ],
        [ 64.80334  ,   2.0496187],
        [ 69.06844  ,   2.2629347],
        [ 73.93755  ,   2.3781548],
        [ 76.02232  ,  15.951238 ],
        [ 76.04037  ,   2.5013251],
        [ 77.79303  ,   2.0136585],
        [ 81.07086  ,   2.6381946],
        [ 82.37904  ,   2.6284096],
        [ 82.38445  ,   2.2936356],
        [ 83.08619  ,   4.535777 ],
        [ 84.04508  ,  64.60917  ],
        [ 85.10181  ,  18.420761 ],
        [ 86.99043  ,  13.63216  ],
        [100.11194  ,   2.446773 ],
        [101.96172  ,   2.1557875],
        [102.055    ,   2.991438 ],
        [105.00076  ,   3.9253068],
        [112.0397   ,   9.185551 ],
        [116.01683  , 174.01921  ],
        [127.09774  ,   2.2379827],
        [129.0

In [89]:
df[df['SPECTRUMID'] == 'CCMSLIB00010121918']

,PEPMASS,SPECTRUMID,SCANS,peaks,CHARGE,MSLEVEL,SOURCE_INSTRUMENT,FILENAME,SEQ,IONMODE,...,NAME,PI,DATACOLLECTOR,SMILES,INCHI,INCHIAUX,PUBMED,SUBMITUSER,LIBRARYQUALITY,PRECURSOR_MZ
19897,426.079,CCMSLIB00010121918,19483,"[[104.04935, 0.9985562], [105.06989, 1.6460254...",1,2,LC-ESI-Orbitrap,tharwood/spectral_libraries/BERKELEY-LAB.mgf,*..*,Positive,...,N-(4-chlorophenyl)-2-[9-(4-methylphenyl)-6-oxo...,Trent Northen,JGI,Cc1ccc(-n2c(SCC(=O)Nc3ccc(Cl)cc3)nc3c(O)ncnc32...,"""InChI=1S/C20H16ClN5O2S/c1-12-2-8-15(9-3-12)26...",N/A,N/A,mwang87,3,426.079


In [ ]:
library_mgf[19897]

{'PEPMASS': 426.079,
 'SPECTRUMID': 'CCMSLIB00010121918',
 'SCANS': '19483',
 'peaks': array([[1.0404935e+02, 9.9855620e-01],
        [1.0506989e+02, 1.6460254e+00],
        [1.1806535e+02, 1.3347471e+00],
        [1.2524415e+02, 9.0558362e-01],
        [1.3006508e+02, 1.1683723e+01],
        [1.3107291e+02, 1.4374791e+00],
        [1.3305276e+02, 9.6902072e-01],
        [1.3502632e+02, 1.4369655e+00],
        [1.4002617e+02, 1.2261078e+00],
        [1.4507567e+02, 5.0477428e+00],
        [1.4802115e+02, 1.0807419e+00],
        [1.4807570e+02, 3.6757553e+00],
        [1.4962625e+02, 7.8821218e-01],
        [1.5003720e+02, 4.7919397e+00],
        [1.5707527e+02, 7.1658220e+00],
        [1.5796132e+02, 8.8991696e-01],
        [1.5805939e+02, 8.9906015e+00],
        [1.5808392e+02, 1.2777307e+00],
        [1.5906776e+02, 5.7026601e+00],
        [1.6007632e+02, 1.2076161e+00],
        [1.6203699e+02, 7.7996796e+01],
        [1.6304526e+02, 1.1850150e+00],
        [1.6405270e+02, 1.0943026e

In [96]:
spectrum_mgf[41]

{'PEPMASS': 434.128,
 'SPECTRUMID': 'CCMSLIB00010124534',
 'SCANS': '22098',
 'peaks': array([[5.0090961e+01, 1.0042304e+00],
        [6.0774689e+01, 1.0781515e+00],
        [6.5412918e+01, 1.2108222e+00],
        [6.7712196e+01, 1.0006795e+00],
        [7.3937401e+01, 1.0300543e+00],
        [9.7054321e+01, 1.1549778e+00],
        [1.0305491e+02, 9.9293232e-01],
        [1.0507028e+02, 2.0065238e+00],
        [1.3006531e+02, 1.3763863e+01],
        [1.3107298e+02, 1.3079849e+00],
        [1.3502661e+02, 1.4793917e+00],
        [1.4507605e+02, 5.9763165e+00],
        [1.4807545e+02, 6.3746514e+00],
        [1.5003694e+02, 5.0033975e+00],
        [1.5707594e+02, 5.9772849e+00],
        [1.5806006e+02, 1.0315715e+01],
        [1.5808347e+02, 1.7337580e+00],
        [1.5906750e+02, 5.2806826e+00],
        [1.6007629e+02, 1.3564049e+00],
        [1.6203705e+02, 7.3391129e+01],
        [1.6405296e+02, 1.0123649e+01],
        [1.6587277e+02, 1.1204382e+00],
        [1.6907585e+02, 4.6676860e

In [85]:
calculate_exact_score_GNPS(spectrum_list[41], library_list[22511], 0.5)

(0.9879885557747912, 0.9834920949360821, 0.004496460838709027, 47.0)

In [90]:
calculate_exact_score_GNPS(spectrum_list[41], library_list[19897], 0.5)

(0.9938496128888801, 0.9898371418239549, 0.004012471064925194, 47.0)

In [26]:
cand

(24925, 0.9660447175847366, 0.9660447175847366, 0.0, 21.0)

In [97]:
df1 = pd.read_csv("/home/user/LabData/Reza/Code/LibrarySearch_Workflow/bin/NextflowModules/bin/library_search/temp/34f610f3-39a7-35a8-aa4e-0db4017fda8d:__xianghu_tsv_rest.mgf.tsv", sep="\t")
df2 = pd.read_csv("/home/user/LabData/Reza/Code/LibrarySearch_Workflow/nf_output/merged_results.tsv", sep="\t")

In [102]:
df1[['#Scan#', 'LibrarySpectrumID']].merge(
    df2[['#Scan#', 'LibrarySpectrumID']],
    on=['#Scan#', 'LibrarySpectrumID'],
    how='outer'
)

,#Scan#,LibrarySpectrumID
0,22098,CCMSLIB00010121918
1,21787,CCMSLIB00010123542
2,2908,CCMSLIB00010107847
3,21219,CCMSLIB00010123751
4,14544,CCMSLIB00010119560
...,...,...
57,3646,CCMSLIB00010114606
58,9218,CCMSLIB00010108213
59,9883,CCMSLIB00010108213
60,21210,CCMSLIB00010103108
